<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Localisation-des-services-les-plus-proches-d'un-bien-immobilier" data-toc-modified-id="Localisation-des-services-les-plus-proches-d'un-bien-immobilier-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Localisation des services les plus proches d'un bien immobilier</a></span><ul class="toc-item"><li><span><a href="#La-base-BPE" data-toc-modified-id="La-base-BPE-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>La base BPE</a></span><ul class="toc-item"><li><span><a href="#Présentation-de-la-base" data-toc-modified-id="Présentation-de-la-base-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Présentation de la base</a></span></li><li><span><a href="#Choix-des-modalités-retenues-et-des-regroupements-effectués" data-toc-modified-id="Choix-des-modalités-retenues-et-des-regroupements-effectués-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Choix des modalités retenues et des regroupements effectués</a></span></li></ul></li><li><span><a href="#Calcul-de-la-distance-aux-services-le-plus-proches" data-toc-modified-id="Calcul-de-la-distance-aux-services-le-plus-proches-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calcul de la distance aux services le plus proches</a></span><ul class="toc-item"><li><span><a href="#Lecture-des-données" data-toc-modified-id="Lecture-des-données-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Lecture des données</a></span><ul class="toc-item"><li><span><a href="#Données-pour-identifier-les-équipements" data-toc-modified-id="Données-pour-identifier-les-équipements-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>Données pour identifier les équipements</a></span></li><li><span><a href="#Données-concernant-les-positions-des-mutations" data-toc-modified-id="Données-concernant-les-positions-des-mutations-1.2.1.2"><span class="toc-item-num">1.2.1.2&nbsp;&nbsp;</span>Données concernant les positions des mutations</a></span></li></ul></li><li><span><a href="#Filtre-et-mise-en-forme" data-toc-modified-id="Filtre-et-mise-en-forme-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Filtre et mise en forme</a></span></li><li><span><a href="#Calcul-de-la-distance-au-plus-proche" data-toc-modified-id="Calcul-de-la-distance-au-plus-proche-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Calcul de la distance au plus proche</a></span></li></ul></li><li><span><a href="#Application-de-la-fonction" data-toc-modified-id="Application-de-la-fonction-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Application de la fonction</a></span></li></ul></li></ul></div>

# Localisation des services les plus proches d'un bien immobilier

Le but de ce notebook est de proposer un moyen d'identifier les services à proximité d'un bien. En effet, on fait ici l'hypothèse que la valeur d'un bien est également fonction de la proximité aux services (alimentaire, santé, sécurité ...)

## La base BPE

### Présentation de la base

La table `bpe19_ensemble_xy`est stocké dans le dossier `gooddata/data` car il fait plus de 150 Mo.

### Choix des modalités retenues et des regroupements effectués

## Calcul de la distance aux services le plus proches

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely import geometry
from scipy.spatial import cKDTree
from shapely.geometry import Point
import numpy as np

dossier = 'C:/Users/khale/OneDrive/Documents/Ensae/Python pour le data scientist/Projets/dvf_ensae_sbra/'

### Lecture des données

#### Données pour identifier les équipements

In [2]:
donnees_bpe = pd.read_csv(dossier + 'good_data/data/bpe/bpe19_ensemble_xy.csv', sep = ";")
lib_bpe = pd.read_csv(dossier + '/good_data/data/bpe/varmod_bpe19_ensemble_xy.csv', sep = ";")
bpe_a_garder = pd.read_csv(dossier + '/good_data/data/bpe/bpe_a_garder.csv', sep = ";", header = None)

C:\Users\khale\anaconda3\envs\projet_2A\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Données concernant les positions des mutations

In [3]:
mutation = gpd.read_file(dossier + 'good_data/data/filo_mut_merge/paris_mutation_filo.shp')
mutation.set_crs(epsg=4326, inplace=True)

,idmutation,datemut,valeurfonc,nblot,l_codinsee,nbpar,nblocmut,nblocdep,sbati,pp,...,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,ind_par_zo,nv_par_hab,ind_par__1,geometry
0,5454034,2017-03-07,207000.0,3,{75118},1,1,0,39,3,...,1.465739,0.893211,0.411347,0.369373,0.142712,0.060443,19882.503871,25609.782463,549.67,POINT (2.34927 48.89034)
1,5183819,2019-11-27,220000.0,2,{75115},1,1,0,21,2,...,2.373654,1.262352,0.895515,0.976435,0.393811,0.156445,12101.089514,37578.001910,387.83,POINT (2.31513 48.84359)
2,5186543,2019-07-04,455000.0,1,{75118},1,1,0,61,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,5603750,2016-11-30,220000.0,1,{75116},1,1,0,25,1,...,1.643457,1.935802,1.208889,1.058765,0.410864,0.371358,13923.583665,47830.614933,462.89,POINT (2.28300 48.86192)
4,5340190,2018-08-30,295000.0,1,{75115},1,1,0,29,1,...,2.116317,2.711532,1.415287,1.732735,0.978797,0.251313,10496.383220,35538.059919,306.55,POINT (2.28448 48.84956)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178909,5630439,2016-06-30,280000.0,1,{75110},1,1,0,36,2,...,2.471176,1.447344,0.583872,0.505748,0.127465,0.065788,16761.393570,28326.362103,360.83,POINT (2.35211 48.87021)
178910,5871270,2014-02-13,145000.0,1,{75113},1,1,0,16,1,...,1.938971,1.981680,1.247092,1.366676,0.478336,0.316044,4476.398601,29306.983179,284.50,POINT (2.36955 48.83017)
178911,5854812,2014-02-14,50000.0,2,{75108},1,2,1,33,1,...,14.311057,20.870291,10.733293,6.559234,4.174058,1.192588,1181.134328,46488.064865,49.46,POINT (2.32473 48.86849)
178912,5306485,2018-09-07,662000.0,2,{75117},1,1,0,54,3,...,2.304429,1.059265,0.575230,0.547170,0.231495,0.108733,12137.178609,37195.265857,454.79,POINT (2.32029 48.88559)


Le code que nous utilisons pour déterminer la distance au plus proche service utilise les coordonnées en Lambert 93 : on va créer une colonne `LAMBERT` contenant cette information (un tuple de la forme *(x,y*)).

On divise la table `mutation` en deux : 
- `mutation_avec_coord` : l'ensemble des mutations ayant une position renseignée
- `mutation_sans_coord` : le complémentaire

car la fonction `to_crs` ne fonctionne pas sur des objets de type `None`.

In [4]:
mutation_avec_coord = mutation[mutation["geometry"] != None]
mutation_sans_coord = mutation[mutation["geometry"] == None]

In [5]:
mutation_avec_coord["lambert"] = mutation_avec_coord["geometry"].to_crs({'init' :"EPSG:2154"})

mutation_avec_coord["lambert"] = mutation_avec_coord.apply(lambda row : (row["lambert"].x,row["lambert"].y), axis = 1)

C:\Users\khale\anaconda3\envs\projet_2A\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\khale\anaconda3\envs\projet_2A\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [6]:
mutation_sans_coord["lambert"] = None

### Filtre et mise en forme

In [7]:
donnees_bpe["QUALITE_XY"].value_counts()

donnees_bpe_paris = donnees_bpe[donnees_bpe["DEP"] == "75"]

donnees_bpe_paris_loc = donnees_bpe_paris[donnees_bpe_paris["QUALITE_XY"].isin(["Bonne","Acceptable"])]

In [8]:
#Récupération des libellésc
lib_typequ = lib_bpe[lib_bpe["COD_VAR"] == "TYPEQU"][["COD_MOD", "LIB_MOD"]]

donnees_bpe_paris_lab = donnees_bpe_paris_loc.merge(lib_typequ, left_on = "TYPEQU", right_on = "COD_MOD", how = "left")

#Il  existe une nomenclature
donnees_bpe_paris_lab["SOUSDOM"] = donnees_bpe_paris_lab["TYPEQU"].str[:2]


In [9]:
bpe_a_garder.rename(columns={0 : "COD_MOD", 1 : "LIB_MOD"}, inplace = True)

bpe_a_garder_sousdom = bpe_a_garder[bpe_a_garder["COD_MOD"].str.len() == 2]
bpe_a_garder_mod =  bpe_a_garder[bpe_a_garder["COD_MOD"].str.len() == 4]

In [10]:
donnees_bpe_paris_finale = donnees_bpe_paris_lab[donnees_bpe_paris_lab["COD_MOD"].isin(bpe_a_garder_mod["COD_MOD"]) | donnees_bpe_paris_lab["COD_MOD"].isin(bpe_a_garder_sousdom["COD_MOD"])]

donnees_bpe_paris_finale = donnees_bpe_paris_finale.merge(bpe_a_garder, on = "COD_MOD", how = "left")

#service est une Series
services_retenus = donnees_bpe_paris_finale["LIB_MOD_y"].drop_duplicates()


### Calcul de la distance au plus proche

In [11]:
services_retenus = donnees_bpe_paris_finale["LIB_MOD_y"].drop_duplicates()

#points_a_tester : une liste contenant des tuples (x,y) désignant la position du point qu'on étudie en LAMBERT 93
#service : nom d'un regroupement de service (modalité de la variable LIB_MOD_Y de la la table donnees_bpe_)
def nearest_service(points_a_tester, service, donnees = donnees_bpe_paris_finale): 
    points_a_tester = np.array(points_a_tester)
    points_services = list(donnees_bpe_paris_finale[donnees["LIB_MOD_y"] == service].apply(lambda x: (x.LAMBERT_X, x.LAMBERT_Y), axis = 1))
    points_services = np.array(points_services)
    btree = cKDTree(points_services)
    dist, idx = btree.query(points_a_tester, k=1)
    res = pd.Series(dist).to_frame(service)
    return res


def nearest_services(point, services = services_retenus, donnees = donnees_bpe_paris_finale):
    minimal_dist_to_services = list(map(lambda x : nearest_service(point,x,donnees), services_retenus))
    res = pd.concat(minimal_dist_to_services, axis = 1)
    return(res)

## Application de la fonction

In [12]:
service_proche_mut = nearest_services(list(mutation_avec_coord["lambert"]), services_retenus)

In [13]:
service_proche_mut["idmutation"] = mutation_avec_coord["idmutation"]
mutation_filo_service_avec_coord = mutation_avec_coord.merge(service_proche_mut, on = "idmutation")

In [14]:
mutation_filo_servpd.concat([mutation_filo_service_avec_coord,mutation_sans_coord])

,idmutation,datemut,valeurfonc,nblot,l_codinsee,nbpar,nblocmut,nblocdep,sbati,pp,...,BOULANGERIE,BOUCHERIE,SUPERETTE,POLICE,POSTE,CINÉ,BIBLIOTHÈQUE,MUSÉE,CRÈCHE,THÉATRE
0,5454034,2017-03-07,207000.0,3,{75118},1,1,0,39,3,...,33.948004,16.921520,140.927084,111.076835,185.908396,762.429072,376.997387,681.897347,207.691701,201.465435
1,5183819,2019-11-27,220000.0,2,{75115},1,1,0,21,2,...,72.812387,192.941457,153.915358,987.630695,286.365726,705.769603,753.843452,217.046814,39.422980,625.757503
2,5603750,2016-11-30,220000.0,1,{75116},1,1,0,25,1,...,106.266384,360.891324,257.182392,296.749253,329.470154,151.534903,122.578228,698.112301,132.773034,745.411484
3,5340190,2018-08-30,295000.0,1,{75115},1,1,0,29,1,...,43.464850,49.308220,80.522445,712.489527,238.897209,619.979388,675.106825,1672.584901,126.097987,502.697552
4,5192801,2019-06-20,678000.0,2,{75115},1,1,0,69,3,...,82.122440,47.326482,167.094580,213.976279,208.304484,668.085681,464.184792,1533.057728,35.873257,462.968391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169971,5181646,2019-08-02,64500.0,1,{75116},2,1,0,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170841,5476502,2017-05-09,518465.0,3,{75117},1,2,1,47,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173103,5455338,2017-04-21,1073500.0,4,{75117},1,2,1,150,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175483,5167233,2019-01-30,449000.0,1,{75118},1,1,0,61,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
